In [2]:
import os
os.getcwd()
os.chdir('c:\\Users\\Stior\\Documents\\cours\\L3SID\\Projet-InterPromo\\ProjetInterpromo2020G8')

In [59]:
from pipeline import Pipeline, Process, Postprocess, Preprocess
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import PIL

In [4]:
class MyPreProcess(Preprocess):
    process_desc = "Exemple de pre-process -> ne fait rien"
    def run(self, images):
        pass


class MyProcess(Process):
    process_desc = "Exemple de process -> ne fait rien"
    def run(self, images):
        pass
    
    
class MyPostProcess(Postprocess):
    process_desc = "Exemple de post-process -> ne fait rien"
    def run(self, images):
        pass

In [5]:
pipeline = Pipeline()
process_array = np.array([MyPreProcess()
                          ,MyProcess()
                          ,MyPostProcess()
                          ,MyPreProcess(verbose=0)])
pipeline.add_processes(process_array)


Preprocess :  Exemple de pre-process -> ne fait rien
Process :  Exemple de process -> ne fait rien
Postprocess :  Exemple de post-process -> ne fait rien
Exemple de pre-process -> ne fait rien a été ajouté.
Exemple de process -> ne fait rien a été ajouté.
Exemple de post-process -> ne fait rien a été ajouté.
Exemple de pre-process -> ne fait rien a été ajouté.


In [6]:
pipeline.print_process()

Exemple de pre-process -> ne fait rien
Exemple de pre-process -> ne fait rien
Exemple de process -> ne fait rien
Exemple de post-process -> ne fait rien


In [7]:
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 1)

#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A320_plane9.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A320_plane9.jpg', 1)


#img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Air_Transat_Airbus_A330-300_V2_plane128.jpg', 0)
#img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Air_Transat_Airbus_A330-300_V2_plane128.jpg', 1)

#template = cv2.imread('./images/temp_normal_seat.jpg', 0)
template = cv2.imread('./images/templateTest.jpg', 0)
#template = cv2.imread('./images/templateNormalSeat-SansBlanc.png', 0)
#template = cv2.imread('./images/templateNormalSeat-SansBlanc.jpg', 0)

#cv2.imshow('Test', img2)
#cv2.waitKey()
#cv2.destroyAllWindows() 


h, w = template.shape


In [9]:

methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

In [11]:

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
threshold = 0.7
loc = np.where( res >= threshold)

coord = []

for pt in zip(*loc[::-1]):
    coord.append((pt[0], pt[1]))
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

for i in coord:
    cv2.circle(img_rgb, i, 1, (255, 0, 0), -1)

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows() 

print(coord)


with open("coord.txt", "a") as fichier:
    for xy in coord:
        fichier.write(str(xy[0]) + ", " + str(xy[1]) + "\n")



[(146, 517), (133, 537), (146, 537), (185, 537), (198, 537), (211, 537), (224, 537), (263, 537), (276, 537), (133, 557), (146, 557), (185, 557), (198, 557), (211, 557), (224, 557), (263, 557), (276, 557), (133, 577), (146, 577), (185, 577), (198, 577), (211, 577), (224, 577), (263, 577), (276, 577), (133, 597), (146, 597), (185, 597), (198, 597), (211, 597), (224, 597), (263, 597), (276, 597), (133, 617), (146, 617), (185, 617), (198, 617), (211, 617), (224, 617), (263, 617), (276, 617), (133, 637), (146, 637), (185, 637), (198, 637), (211, 637), (224, 637), (263, 637), (276, 637), (133, 657), (146, 657), (185, 657), (198, 657), (211, 657), (224, 657), (263, 657), (276, 657), (133, 677), (146, 677), (185, 677), (198, 677), (211, 677), (224, 677), (263, 677), (276, 677), (133, 697), (146, 697), (185, 697), (198, 697), (211, 697), (224, 697), (263, 697), (276, 697), (133, 717), (146, 717), (185, 717), (198, 717), (211, 717), (224, 717), (263, 717), (276, 717), (133, 737), (146, 737), (18

In [29]:
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 1)
#template = cv2.imread('./images/temp_normal_seat.jpg', 0)
templates = ['./images/templateBusinessSeatLeft.jpg', './images/templateBusinessSeatRight.jpg', './images/templateBusinessElectricity.jpg']
#templates = ['./images/templateBusinessElectricity.jpg']

In [31]:
for templ in templates:
    template = cv2.imread(templ, 0)
    h, w = template.shape[::]
    
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    
    if templ == './images/templateBusinessElectricity.jpg':
        threshold = 0.7
    else:
        threshold = 0.7

    loc = np.where( res >= threshold)

    coord = []

    fichier = open("coord.txt", "a")

    if templ == './images/templateBusinessElectricity.jpg':
        fichier.write('-------------------\nPrise electrique \n')

    for pt in zip(*loc[::-1]):
        fichier.write(str(pt[0]) + ", " + str(pt[1]) + "\n")
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
    fichier.close()

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()
#cv2.imwrite('test.jpeg', img_rgb)

print(np.where( res >= threshold))

(array([334, 334, 334, 334, 334, 367, 367, 368, 399, 401, 431, 431, 435,
       435, 435], dtype=int64), array([140, 190, 217, 268, 269, 191, 218, 250, 190, 140, 191, 218, 149,
       250, 277], dtype=int64))


In [83]:
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 1)
#template = cv2.imread('./images/temp_normal_seat.jpg', 0)
templates = ['./images/templateBusinessElectricity.jpg', './images/templateNormalSeat-SansBlanc.jpg', './images/templateTest.jpg', './images/templateBusinessSeat.jpg', 'templateTest.jpg']



In [87]:
templateTest = PIL.Image.open(templates[3])
rotate = templateTest.rotate(45)
rotate.show()
rotate.save('templateTest.jpg')

In [86]:
for templ in templates:
    template = cv2.imread(templ, 0)
    h, w = template.shape[::]
    
    res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
    
    threshold = 0.8
    
    if templ == templates[0]:
        threshold = 0.8
    if templ == templates[1]:
        threshold = 0.8
    if templ == templates[2]:
        threshold = 0.8
    if templ == templates[3]:
        threshold = 0.8
    if templ == templates[4]:
        threshold = 0.4


    loc = np.where( res >= threshold)

    coord = []

    #fichier = open("coord.txt", "a")

    for pt in zip(*loc[::-1]):
        #fichier.write(str(pt[0]) + ", " + str(pt[1]) + "\n")
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    
    #fichier.close()

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('test.jpeg', img_rgb)

print(np.where( res >= threshold))

(array([  34,   34,   35, ..., 1772, 1772, 1772], dtype=int64), array([198, 199, 197, ..., 180, 181, 182], dtype=int64))
